In [2]:
from ipywidgets import interactive, widgets, fixed
from surfer import Brain as surface
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import minmax_scale

import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# spectrome:
from spectrome.brain import Brain
from spectrome.utils import functions, path

In [3]:
%gui qt

# pysurfer variables:
subject_id = 'fsaverage'
hemi = ['lh', 'rh']
surf = 'white'

# parcellation:
hemi_side = 'lh'
aparc_file = os.path.join(os.environ['SUBJECTS_DIR'],
                         subject_id, 'label',
                         hemi_side + '.aparc.annot')
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

- Introduction:
     - structure function problem
     - modalities of data
     - connectivity
     - mean field models
     
 - DWI advances
 - NMM brief explanation and results
 - complex laplacian
 - spectral graph model

<p style = "text-align:center; font-size:150%;">Model Based Analysis of Multimodal Neuroimaging: From Neural Masses to Spectral Graph Theory</p>
<p style = "text-align:center; font-size:70%;"><em>Thesis Defense</em></p>
<p style = "text-align:center; font-size:70%;"><em>Presented to the Weill Cornell Graduate School of Medical Sciences</em></p>
<p style = "text-align:center; font-size:70%;"><em>In Fulfillment of the Requirements for the Degree of Doctor of Philosophy</em></p>

<center>
<img src = "figures/presentation/whole_brain.png" alt = "intro" width = "400" style = "align:center"/>
</center>

<p style = "text-align:center; font-size:70%;">Xi He Xie</p>
<p style = "text-align:center; font-size:70%;">Department of Neuroscience, Weill Cornell Medicine</p>
<p style = "text-align:center; font-size:70%;">Visiting Graduate, University of California San Francisco</p>
<p style = "text-align:center; font-size:70%;">Nov 8th, 2021</p>

What's so special about the human brain?
---

 - Humans have the remarkable ability to invent symbol systems such as the arabic numeric system or the alphabet.

<center>
    <img src = "figures/presentation/lost_data.png" alt = "messy_data" width = "900" style = "align:center" />
</center>

<aside class="notes">
Before I start talking in jargons and data, let's think for a second. I'm sure everyone here had days like this guy, we are constantly reading papers, crunching data, doing numerical analysis, and it can get extremely messy if you don't practice good organizational routines. Despite all that, our brains are still efficiently taking these symbols, putting them into context, and processes the complicated information... if you had enough coffee
</aside>

#### Exercise:

 - What is so special about human brain functionality that allows it to acquire these cultural tools? (Language, reading, arithmetics) Let's examine the two possible hypothesis:

 1) Cultural functionality is acquired by expanded cortical plasticity unique to humans.
     

 2) Humans have evolved new specialized circuitry, each providing new cognitive functions.

<aside class="notes">
On one extreme, you can have (1), on the other you can have (2). So very quickly, by a show of hands, who agrees with 1? la la la... who agrees with (2)?

 - Human brians accomodate a broad range of new functions through learning.
    
 - At the extreme, the brain's architecture exerts little to no contraints on the range of competence we can acquire.
    
 - This may also suggest individual brain's implementation of cultural abilities is highly variable across individuals.
    
 - Humans experienced evolutionary pressure to develop brain circuitry for understanding symbols?
    
 - But reading was invented ~5000 years ago, and artihmetics even more recent!
</aside>

 - Primates have prior brain functions and structures that were eventually adapted and efficiently used by humans.
 
<!-- <center>
    <img src = "figures/presentation/xkcd_surgery.png" alt = "xkcd" width = "1000" style = "align:center" />
<center/>
<aside class="notes">
So this is from xkcd, and it's true there's an xkcd comic for everything. The brain's structure itself stays the same, but when our ancestors acquire new tools like symbols in arithmetics and reading, their brains adapted what was already available in primate brains and expand it's capabilities like a software update.
</aside> -->

### The structure-function problem in neuroscience:
The brain's structure at various spatial scales is critical for determining its function.

<b>My thesis' interest in particular: Finding the relationship between the brain's <em>structural wiring</em> and the <em>functional</em> patterns of neural activity.</b>
 - Consider the brain to be a macroscopic network of nodes and edges:
     - Identifiable gray matter brain regions of interest (ROIs)
     - Long range connections between brain regions via white matter fiber bundles

<center>
    <img src = "figures/presentation/brain_nodes_edges.png" alt = "braingraph" width = "400" style = "align:right"/>
</center>

Now we have a problem:
--- 
<center>
    <img src = "figures/presentation/xkcd_brain.png" alt = "messy_data" width = "1000" style = "align:center" />
</center>
<p style = "text-align:center; font-size:75%;"> xkcd-1163</p>
 
 - We want to look at the interesting stuff going on inside a living human brain...
 - What are some non-invasive methods of recording data from a living human brain?

#### We can measure brain data non-invasively in humans with ...

# functional data:
### 1) Human Magnetoencephalography (MEG) frequency spectrum:
<center>
    <img src = "figures/presentation/meg_spectrum_example.png" alt = "actualmeg" width = "1200" style = "align:center"/>
</center>
<aside class="notes">
The first functional human brain data we collect is called MEG. Where we have our subjects sit inside an expensive insulated room under an even more expensive helmet. The helmet is cooled to near absolute zero to cool down the SQUIDS (superconducting quantum interference devices), these sensors can detect subtle changes in magnetic field as induced by evoked currents from the neurons. Anyone remember the right hand rule from physics class?
    
The result from each sensor is then mapped to our brain regions of interest, but we don't like looking at it over time, because all we would see are squiggly oscillatory lines. So we decompose these oscillations into combinations of sin/cos waves of various frequencies and power, and plot them here in what we call a power spectrum. Humans have very replicable and similar power spectrums, where we have idenfiable delta, theta, alpha, beta, gamma waves. The alpha wave usually stands out the most, if everyone closed their eyse right now, the 10 Hz signal in the posterior regions of your brain would just light up! And beta band is often mentioned in motor tasks with beta suppresion at the lateral motor cortex
</aside>

<center>
    <img src = "figures/presentation/fmri_modalities.png" alt="fmri" width="1300" style="align:center" />
</center>

<aside class="notes">
Structural: The hydrogen molecules in our bodies contains 1 positive charge (protons), like the earth spinning on its axis with magnetic poles, each spinning charged molecule is also like a tiny magnet that spins around it's own axis. MRI uses magnetic fields and radio waves to measure locations of these molecules. These protons are all in random directions, the MRI machine uses a magnet that ranges from 1-7 Teslas to align all our protons in specific directions. For reference, the earth's magnetic field is ~0.6e-4T. The scanners produce a magnetic field that's 60,000 times stronger! 
fMRI: In our brains, we have billions of neurons, when a lot of these neurons begin to fire, the body rushes in oxygen to help supply energy. This is called a hemodynamic response, and what we measure is called blood oxygen level dependent signal (BOLD). 
DWI: The brain is connected by long distance projections called axonal fiber tracts. We specifically track the movement of water molecules inside the brain. And build a gradient of water diffusion. 
</aside>

### Structural data: diffusion weighted imaging
<center>
    <img src = "figures/presentation/tract_viz.gif" alt = "dti" width = "1200" style = "align:center"/>
</center>

# Connectivity - Anatomical

#### Quantify all the traced white matter streamlines between all brain regions into a matrix (Structural connectivity)

<center>
<img src = "figures/presentation/SC-example.png" alt = "SCmatrix" width = "1000" style = "align:center"/>
</center>
<aside class="notes">
Now we align both the white matter fibers and our anatomical MRI scans, from each of these ROIs, we can start counting the number of fibers going to each of the other connected regions, and summarize all the connections in this connectivity matrix. The brain is what we like to call a small world network, where there are dense local connections for specialized function, and long range connections for integration of function with other hubs of connections. Just the inter- and intra- hemispheric connections you see here, dense within hemisphere, sparse across hemispheres.
</aside>

# Connectivity - Functional

Network neuroscience described in 3 dimensions by Bassett, Zurn & Gold's review in 2018:

<center>
<img src = "figures/presentation/dani_dimensions.png" alt = "dims" width = "900" style = "align:center"/>
</center>

## Simpson's Paradox:

<center>
<img src = "figures/chapter1/simpson.png" alt = "simpson" width = "1000" style = "align:center"/>
</center>

## Functional Connectivity

<center>
<img src = "figures/presentation/fc_metrics.png" alt = "FC" width = "1000" style = "align:center"/>
</center>

Canonical Functional Networks
---
 - Obtained from resting-state fMRI recordings
 - Consistently observed functional networks in both resting-state fMRI and MEG
 - There are seven of these prominent, most frequently visited functional networks identified by neuroimaging studies.

In [5]:
interactive(
    get_fc_values,
    fc_df=fixed(fc_dk_normalized),
    labels=fixed(labels),
    fc_name=widgets.RadioButtons(
        options=fc_names, value="Limbic", description="Select canonical network"
    ),
)

interactive(children=(RadioButtons(description='Select canonical network', index=1, options=('Visual', 'Limbic…

## Measuring brain structure and function:

We have ...
- Resting-state functional recordings from MEG, fMRI functional networks

 - Anatomical magnetic resonance images (MRIs) to co-register MEG coordinates to "head" coordinates

 - Parcellated brain regions from MRI (68 cortical regions and 18 subcortical regions in our atlas of choice)

 - Diffusion weighted imaging (DTI) for anatomical wiring architecture via tractography

 - Hard working graduate students!

# Modeling FC with Mean Field Theory in Neuroscience: Neural Mass Models

Spatial Patterns in Alpha-band:

<center>
<img src = "figures/presentation/SGM_alpha_spatial.png" alt = "sg7" width = "1000" style ="align:center"/>
</center>

Spatial Patterns in Beta-band:

<center>
<img src = "figures/presentation/SGM_beta_spatial.png" alt = "sg7" width = "1000" style ="align:center"/>
</center>

Spectral graph model performance:
<center>
    <img src = "figures/presentation/SGM_spatial_violin.png" alt = "model_comparison" width = "1200" style = "align:center"/>
</center>